<a href="https://colab.research.google.com/github/resc863/Kakao_Chatbot/blob/master/meal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# URL Example

http://stu.pen.go.kr/sts_sci_md01_001.do?schulCode=C100000521&schulCrseScCode=4&schulKndScCode=04&schMmealScCode=2&schYmd=2019.11.04

# API

API(Application Programming Interface, 응용 프로그램 프로그래밍 인터페이스)는 응용 프로그램에서 사용할 수 있도록, 운영 체제나 프로그래밍 언어가 제공하는 기능을 제어할 수 있게 만든 인터페이스를 뜻한다.

오픈 API - 하나의 웹 사이트에서 자신이 가진 기능을 이용할 수 있도록 공개한 프로그래밍 인터페이스

# 파라미터

http://stu.pen.go.kr/sts_sci_md01_001.do -> URL

파라미터

*   ?schulCode=C100000521
*   &schulCrseScCode=4
*   &schulKndScCode=04
*   &schMmealScCode=2
*   &schYmd=2019.11.04

| 파라미터 | 기능 | 예시 |
|--|--|--|
|`schulCode`|학교 고유 코드|C100000521|
|`schulCrseScCode`|학교 종류 코드|4 (고등학교)|
|`schMmealScCode`|조식, 중식, 석식 코드|2 (중식)|
|`schYmd`|날짜|2019-11-11|

# 모듈 불러오기

In [0]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import urllib
import requests, re
import json
import datetime
from datetime import date
import time
from copy import copy

In [0]:
def dayCal() :
  y=str(list(time.localtime(time.time()))[0])
  m=str(list(time.localtime(time.time()))[1])
  d=str(list(time.localtime(time.time()))[2])
  h=str(list(time.localtime(time.time()))[3]+9)
  mi=str(list(time.localtime(time.time()))[4])
  cpy,cpm,cpd,cph,cpmi = copy(y),copy(m),copy(d),copy(h),copy(mi)

  if int(h) >24 :
    h = str(int(h)-24)
    if m == '12' and d == '31' :
      y = str(int(y)+1)
      m = '1'
      d = '1'
    elif d == '31' :
      if m == '1' or m == '3' or m == '5' or m == '7' or m == '8' or m == '10' :
        m = str(int(m)+1)
        d = '1'
    elif d == '30' :
      if m == '4' or m == '6' or m == '9' or m == '11' :
        m = str(int(m)+1)
        d = '1'
    elif d == '29' :
      if m == '2' and int(y)%4 == 0 :
        m = '3'
        d = '1'
    elif d == '28' :
      if m == '2' and int(y)%4 != 0 :
        m = '3'
        d = '1'
    else :
      d = str(int(d)+1)

  if h != '2' and h != '5' and h!= '8' and h!= '11' and h != '14' and h != '17' and h != '20' and h != '23' :
    a = int(h)%3
    h=str(int(h)-a-1)
  else :
    if int(mi)<15 :
      h = str(int(h)-3)

  if h == '-1' :
    h = '23'
    y,m,d = cpy,cpm,cpd

  if int(h)<10 :
    h = '0'+h
  if int(m)<10 :
    m = '0'+m
  if int(d)<10 :
    d = '0'+d
  if int(mi)<10 :
    mi = '0'+mi

  return y,m,d,h,mi

# 급식 스크레이핑 함수

In [0]:
def get_diet(code, ymd, weekday):
    schMmealScCode = code #int 1조식2중식3석식
    schYmd = ymd #str 요청할 날짜 yyyy.mm.dd
    if weekday == 5 or weekday == 6: #토요일,일요일 버림
        element = " " #공백 반환
    else:
        num = weekday + 1 #int 요청할 날짜의 요일 0월1화2수3목4금5토6일 파싱한 데이터의 배열이 일요일부터 시작되므로 1을 더해줍니다.
        URL = (
                "http://stu.pen.go.kr/sts_sci_md01_001.do?"
                "schulCode=C100000521&schulCrseScCode=4"
                "&schulKndScCode=04"
                "&schMmealScCode=%d&schYmd=%s" % (schMmealScCode, schYmd)
            )

        #기존 get_html 함수부분을 옮겨왔습니다.
        html = ""
        resp = requests.get(URL)
        if resp.status_code == 200 : #사이트가 정상적으로 응답할 경우
            html = resp.text
        soup = BeautifulSoup(html, 'html.parser')
        element_data = soup.find_all("tr")
        element_data = element_data[2].find_all('td')
        try:
            element = str(element_data[num])
            print(element)

            #filter
            element_filter = ['[', ']', '<td class="textC last">', '<td class="textC">', '</td>', '&amp;', '(h)', '.']
            for element_string in element_filter :
                element = element.replace(element_string, '')
            #줄 바꿈 처리
            element = element.replace('<br/>', '\n')
            #모든 공백 삭제
            element = re.sub(r"\d", "", element)

        #급식이 없을 경우
        except:
            element = " " # 공백 반환
    return element

def print_get_meal(local_date, local_weekday):
        l_diet = get_diet(2, local_date, local_weekday)
        d_diet = get_diet(3, local_date, local_weekday)

        if len(l_diet) == 1:
            
            
            return "급식이 없습니다"
        elif len(d_diet) == 1:
            lunch = local_date + " 중식\n\n" + l_diet
            return lunch
        else:
            lunch = local_date + " 중식\n\n" + l_diet + "\n"
            dinner = local_date + " 석식\n\n" + d_diet
            meal = lunch + dinner
            return meal

# 날짜

**datetime 모듈**

| 클래스 | 내용 |
|--|--|
|`class datetime.date`|그레고리력 날짜|
|`class datetime.time`|시간을 시,분,초,마이크로 초,시간대로 나타냄|
|`class datetime.datetime`|date클래스와 time클래스의 조합으로 년,월,일,시,분,초,마이크로 초, 시간대 정보를 나타냄|

In [46]:
print(date.today())

2019-12-04


In [47]:
print("=================================================")
print("=                                               =")
print("=       양운고등학교 급식 검색 시스템           =")
print("=                                               =")
print("=================================================")


today = date.today()
print(today)
today = str(today).replace("-", ".")
s = today.replace('.', ', ') #한자리수 달인 경우를 해결하기위함
print(s[6:8])

if int(s[6:8]) < 10:
    s = s.replace(s[6:8], s[7:8])

try : 
    whatday = datetime.datetime.now().weekday()
except :
    meal = 'error'
print("\n")
print("검색중...")

meal = print_get_meal(today, whatday)

print("\n")
print(meal)

=                                               =
=       양운고등학교 급식 검색 시스템           =
=                                               =
2019-12-04
12


검색중...
<td class="textC">나물비빔밥(약고추장)5.6.10.13.<br/>방울어묵볶음1.5.6.13.<br/>대만식지파이(양2)5.6.15.16.<br/>김치9.13.<br/>들깨시락국(센)5.6.13.18.<br/></td>
<td class="textC">햄야채볶음밥2.5.6.10.13.<br/>짜장(김)2.5.6.10.13.16.18.<br/>실파계란국-두부X1.5.6.13.<br/>해쉬브라운5.12.13.<br/>김치9.13.18.<br/>꼬들단무지무침13.<br/>초코우유(양2)2.5.<br/></td>


2019.12.04 중식

나물비빔밥(약고추장)
방울어묵볶음
대만식지파이(양)
김치
들깨시락국(센)

2019.12.04 석식

햄야채볶음밥
짜장(김)
실파계란국-두부X
해쉬브라운
김치
꼬들단무지무침
초코우유(양)



# 내일 급식

In [48]:
print("=================================================")
print("=                                               =")
print("=       양운고등학교 급식 검색 시스템           =")
print("=                                               =")
print("=================================================")

y,m,d,h,mi = dayCal()
if m == '12' and d == '31' :
  y = str(int(y)+1)
  m = '1'
  d = '1'
elif d == '31' :
  if m == '1' or m == '3' or m == '5' or m == '7' or m == '8' or m == '10' :
    m = str(int(m)+1)
    d = '1'
elif d == '30' :
  if m == '4' or m == '6' or m == '9' or m == '11' :
    m = str(int(m)+1)
    d = '1'
elif d == '29' :
  if m == '2' and int(y)%4 == 0 :
    m = '3'
    d = '1'
elif d == '28' :
  if m == '2' and int(y)%4 != 0 :
    m = '3'
    d = '1'
else :
  d = str(int(d)+1)
if int(h)<10 :
  h = '0'+h
if int(m)<10 :
  m = '0'+m
if int(d)<10 :
  d = '0'+d
if int(mi)<10 :
  mi = '0'+mi
tomorrow = y+'.'+m+'.'+d

print(tomorrow)

try : 
    whatday = time.localtime().tm_wday+1
    if whatday == 7 :
      whatday = 0
except :
    meal = 'error'
print("\n")
print("검색중...")

meal = print_get_meal(tomorrow, whatday)

print("\n")
print(meal)

=                                               =
=       양운고등학교 급식 검색 시스템           =
=                                               =
2019.12.05


검색중...
<td class="textC">현미밥<br/>쇠고기무국5.6.13.16.18.<br/>돈육두루치기5.6.10.13.<br/>김치9.13.<br/>계란찜(양2)1.<br/>양배추겉절이(센2)5.6.18.<br/></td>
<td class="textC">녹두밥<br/>석박지9.13.<br/>양반김13.<br/>닭개장(센-국x)1.5.6.13.15.18.<br/>새우살호박볶음9.13.<br/>두부구이/양념장(센)5.6.13.18.<br/></td>


2019.12.05 중식

현미밥
쇠고기무국
돈육두루치기
김치
계란찜(양)
양배추겉절이(센)

2019.12.05 석식

녹두밥
석박지
양반김
닭개장(센-국x)
새우살호박볶음
두부구이/양념장(센)

